# Development of raw data downloaders
##  ONS LA to Community Safety Partnership to PFA (December 2019) Lookup in EW

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from src.data.raw import download_data, data_filters
import src.utilities as utils

### Finding dataset ID and parameters using ONS API

In [ ]:
import requests
import logging
import pandas as pd
from typing import Tuple

utils.setup_logging()
config = utils.read_config()

In [47]:
ENDPOINT = "https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LAD24_CSP24_PFA24_EW_LU/FeatureServer/0/query"

In [52]:
params = {
    "where": "1=1",
    "outFields": "LAD24CD,LAD24NM,PFA24CD,PFA24NM",
    "outSR": 4326,
    "f": "json",
}

In [53]:
def request_to_gdf(url: str, query_params: dict) -> tuple[requests.Response, pd.DataFrame]:
    """Send a get request to ArcGIS API & Convert to GeoDataFrame.

    Only works when asking for features and GeoJSON format.

    Parameters
    ----------
    url : str
        The url endpoint.
    query_params : dict
        A dictionary of query parameter : value pairs.

    Returns
    -------
    requests.response
        The response from ArcGIS API server.
    pd.DataFrame
        A DataFrame of the requested data in the records specified by
        the response metadata.

    Raises
    ------
    requests.exceptions.RequestException
        The response was not ok.
    """
    query_params["f"] = "geoJSON"
    response = requests.get(url, params=query_params, timeout=10)
    if response.ok:
        content = response.json()
        records = [feature['properties'] for feature in content['features']]
        return (
            response,
            pd.DataFrame(records))
    else:
        raise requests.RequestException(
            f"HTTP Code: {response.status_code}, Status: {response.reason}"
        )

In [54]:
_, df = request_to_gdf(ENDPOINT, params)
df.head()

,LAD24CD,LAD24NM,PFA24CD,PFA24NM
0,E07000136,Boston,E23000020,Lincolnshire
1,E07000137,East Lindsey,E23000020,Lincolnshire
2,E07000138,Lincoln,E23000020,Lincolnshire
3,E07000139,North Kesteven,E23000020,Lincolnshire
4,E07000140,South Holland,E23000020,Lincolnshire


In [55]:
df

,LAD24CD,LAD24NM,PFA24CD,PFA24NM
0,E07000136,Boston,E23000020,Lincolnshire
1,E07000137,East Lindsey,E23000020,Lincolnshire
2,E07000138,Lincoln,E23000020,Lincolnshire
3,E07000139,North Kesteven,E23000020,Lincolnshire
4,E07000140,South Holland,E23000020,Lincolnshire
...,...,...,...,...
327,W06000020,Torfaen,W15000002,Gwent
328,W06000021,Monmouthshire,W15000002,Gwent
329,W06000022,Newport,W15000002,Gwent
330,W06000023,Powys,W15000004,Dyfed-Powys


## Starting to build into a script

In [56]:
import src.data.raw.ons_ogp_api as ons_ogp_api

In [58]:
ons_ogp_api.main()

2025-07-01 14:14:04,216 - INFO - Successfully retrieved data from ONS Open Geography Portal.
2025-07-01 14:14:04,221 - INFO - Saving...
2025-07-01 14:14:04,269 - INFO - Data successfully saved to data/interim/LA_to_PFA_(December_2024)_Lookup_in_EW.csv
